In [30]:
setwd('~/Downloads/proj/')

library(data.table)
library(edgeR)
library(EDASeq)
library(RUVSeq)
library(affycoretools)
library(ggplot2)
library(DESeq2)
require(dplyr)
require(stringr)

#project directories, and libraries loaded

MM_cell_line = data.frame(fread("HMCL66_HTSeq_GENE_Counts_v2.txt"))
MM_patients = data.frame(fread("MMRF_CoMMpass_IA13a_E74GTF_HtSeq_Gene_Counts.txt"))

#reading and creating data frames from gene count data

head(MM_cell_line)
head(MM_patients)
length(MM_patients[2,])

#checking output head

Sample,GENE_NAME,ALMC1_DJ_p27,ALMC2_DJ_p11,AMO1_DSMZ_p8,ANBL6_DJ2_p13,COLO677_DSMZ_p8,Delta47_JCRB_p16,DP6_DJ_p7,EJM_DSMZ_p9,⋯,RPMI8226_ATCC_p7,SKMM1_p6,SKMM2_DSMZ_p17,U266_ATCC_p8,UTMC2_p6,VP6_DJ_p5,XG1_p5,XG2_p5,XG6_p5,XG7_p6
ENSG00000000003,TSPAN6,2,0,100,2,0,0,2,88,⋯,2,0,0,0,0,0,0,0,2,114
ENSG00000000005,TNMD,1,0,0,0,1,0,0,0,⋯,0,1,0,0,0,0,0,0,0,0
ENSG00000000419,DPM1,3949,4134,4795,8278,2262,2587,3607,2450,⋯,2694,1247,3263,3044,3108,4538,7336,4430,9279,13828
ENSG00000000457,SCYL3,1597,1513,2412,1988,1530,926,1415,573,⋯,665,994,497,1611,1603,1375,1426,893,1540,1660
ENSG00000000460,C1orf112,3072,1488,2193,4091,3208,1170,3614,1618,⋯,1472,1517,1346,2068,3128,1850,2522,2320,793,689
ENSG00000000938,FGR,0,22,701,5,2,4,89,4,⋯,0,125,777,50,1890,2315,1514,5,3776,847


GENE_ID,MMRF_2801_1_BM,MMRF_2718_1_BM,MMRF_2815_1_BM,MMRF_1656_1_BM,MMRF_2461_1_BM,MMRF_2531_2_PB,MMRF_2057_1_BM,MMRF_1910_1_BM,MMRF_2089_2_BM,⋯,MMRF_2379_1_BM,MMRF_1474_1_BM,MMRF_1641_1_BM,MMRF_1978_2_BM,MMRF_2795_1_BM,MMRF_2316_1_BM,MMRF_2606_2_BM,MMRF_2440_1_BM,MMRF_2731_1_BM,MMRF_1355_1_BM
ENSG00000000003,17,999,85,9,801,20,12,16,4,⋯,2,186,102,347,128,17,11,3,18,6
ENSG00000000005,1,0,0,2,3,0,0,0,0,⋯,0,0,0,0,0,0,1,0,0,0
ENSG00000000419,1495,3271,2303,1234,1896,4400,2013,1745,1698,⋯,2459,1269,337,3347,1049,2545,1625,1253,1905,683
ENSG00000000457,231,809,203,1208,425,2793,710,922,853,⋯,734,771,327,392,90,406,857,548,808,393
ENSG00000000460,87,319,31,183,85,929,174,292,837,⋯,267,104,114,126,17,67,196,141,199,55
ENSG00000000938,318,105,119,190,2064,2952,12,351,498,⋯,46,14,303,2007,58,77,1037,11,174,22


[1] 893

In [31]:
ensembl_intersect = intersect(MM_cell_line$Sample, MM_patients$GENE_ID)

#finding genes in common and creating character vector of Ensembl Gene IDs

head(ensembl_intersect)

#checking output head

[1] "ENSG00000000003" "ENSG00000000005" "ENSG00000000419" "ENSG00000000457"
[5] "ENSG00000000460" "ENSG00000000938"

In [32]:
rownames(MM_cell_line) = MM_cell_line$Sample
rownames(MM_patients) = MM_patients$GENE_ID

#assigning rownames as ensembl gene IDs so as to be able to parse by ensembl Gene IDs

head(MM_cell_line)
head(MM_patients)

#checking output head

,Sample,GENE_NAME,ALMC1_DJ_p27,ALMC2_DJ_p11,AMO1_DSMZ_p8,ANBL6_DJ2_p13,COLO677_DSMZ_p8,Delta47_JCRB_p16,DP6_DJ_p7,EJM_DSMZ_p9,⋯,RPMI8226_ATCC_p7,SKMM1_p6,SKMM2_DSMZ_p17,U266_ATCC_p8,UTMC2_p6,VP6_DJ_p5,XG1_p5,XG2_p5,XG6_p5,XG7_p6
ENSG00000000003,ENSG00000000003,TSPAN6,2,0,100,2,0,0,2,88,⋯,2,0,0,0,0,0,0,0,2,114
ENSG00000000005,ENSG00000000005,TNMD,1,0,0,0,1,0,0,0,⋯,0,1,0,0,0,0,0,0,0,0
ENSG00000000419,ENSG00000000419,DPM1,3949,4134,4795,8278,2262,2587,3607,2450,⋯,2694,1247,3263,3044,3108,4538,7336,4430,9279,13828
ENSG00000000457,ENSG00000000457,SCYL3,1597,1513,2412,1988,1530,926,1415,573,⋯,665,994,497,1611,1603,1375,1426,893,1540,1660
ENSG00000000460,ENSG00000000460,C1orf112,3072,1488,2193,4091,3208,1170,3614,1618,⋯,1472,1517,1346,2068,3128,1850,2522,2320,793,689
ENSG00000000938,ENSG00000000938,FGR,0,22,701,5,2,4,89,4,⋯,0,125,777,50,1890,2315,1514,5,3776,847


,GENE_ID,MMRF_2801_1_BM,MMRF_2718_1_BM,MMRF_2815_1_BM,MMRF_1656_1_BM,MMRF_2461_1_BM,MMRF_2531_2_PB,MMRF_2057_1_BM,MMRF_1910_1_BM,MMRF_2089_2_BM,⋯,MMRF_2379_1_BM,MMRF_1474_1_BM,MMRF_1641_1_BM,MMRF_1978_2_BM,MMRF_2795_1_BM,MMRF_2316_1_BM,MMRF_2606_2_BM,MMRF_2440_1_BM,MMRF_2731_1_BM,MMRF_1355_1_BM
ENSG00000000003,ENSG00000000003,17,999,85,9,801,20,12,16,4,⋯,2,186,102,347,128,17,11,3,18,6
ENSG00000000005,ENSG00000000005,1,0,0,2,3,0,0,0,0,⋯,0,0,0,0,0,0,1,0,0,0
ENSG00000000419,ENSG00000000419,1495,3271,2303,1234,1896,4400,2013,1745,1698,⋯,2459,1269,337,3347,1049,2545,1625,1253,1905,683
ENSG00000000457,ENSG00000000457,231,809,203,1208,425,2793,710,922,853,⋯,734,771,327,392,90,406,857,548,808,393
ENSG00000000460,ENSG00000000460,87,319,31,183,85,929,174,292,837,⋯,267,104,114,126,17,67,196,141,199,55
ENSG00000000938,ENSG00000000938,318,105,119,190,2064,2952,12,351,498,⋯,46,14,303,2007,58,77,1037,11,174,22


In [33]:
MM_cell_line = MM_cell_line[ensembl_intersect, -1]
MM_patients = MM_patients[ensembl_intersect, -1]
MM_patient_ids = data.frame(colnames(MM_patients))
MM_patients_new = filter(MM_patient_ids, str_extract(MM_patient_ids[,1], "_[0-9]_") == "_1_")
keepc = colnames(MM_patients) %in% MM_patients_new[,1]
MM_patients = MM_patients[,keepc]

#filtering MM_cell_line and MM_patients to only those genes in common and taking out 
    #Gene name and ensembl ID columns, then combining them

head(MM_cell_line)
head(MM_patients)
head(MM_combined)

#checking output head

,GENE_NAME,ALMC1_DJ_p27,ALMC2_DJ_p11,AMO1_DSMZ_p8,ANBL6_DJ2_p13,COLO677_DSMZ_p8,Delta47_JCRB_p16,DP6_DJ_p7,EJM_DSMZ_p9,FLAM76_JCRB_p10,⋯,RPMI8226_ATCC_p7,SKMM1_p6,SKMM2_DSMZ_p17,U266_ATCC_p8,UTMC2_p6,VP6_DJ_p5,XG1_p5,XG2_p5,XG6_p5,XG7_p6
ENSG00000000003,TSPAN6,2,0,100,2,0,0,2,88,4,⋯,2,0,0,0,0,0,0,0,2,114
ENSG00000000005,TNMD,1,0,0,0,1,0,0,0,0,⋯,0,1,0,0,0,0,0,0,0,0
ENSG00000000419,DPM1,3949,4134,4795,8278,2262,2587,3607,2450,1388,⋯,2694,1247,3263,3044,3108,4538,7336,4430,9279,13828
ENSG00000000457,SCYL3,1597,1513,2412,1988,1530,926,1415,573,1013,⋯,665,994,497,1611,1603,1375,1426,893,1540,1660
ENSG00000000460,C1orf112,3072,1488,2193,4091,3208,1170,3614,1618,1507,⋯,1472,1517,1346,2068,3128,1850,2522,2320,793,689
ENSG00000000938,FGR,0,22,701,5,2,4,89,4,2,⋯,0,125,777,50,1890,2315,1514,5,3776,847


,MMRF_2801_1_BM,MMRF_2718_1_BM,MMRF_2815_1_BM,MMRF_1656_1_BM,MMRF_2461_1_BM,MMRF_2057_1_BM,MMRF_1910_1_BM,MMRF_2341_1_BM,MMRF_2501_1_BM,MMRF_1383_1_BM,⋯,MMRF_1731_1_BM,MMRF_2799_1_BM,MMRF_2379_1_BM,MMRF_1474_1_BM,MMRF_1641_1_BM,MMRF_2795_1_BM,MMRF_2316_1_BM,MMRF_2440_1_BM,MMRF_2731_1_BM,MMRF_1355_1_BM
ENSG00000000003,17,999,85,9,801,12,16,43,594,693,⋯,1,49,2,186,102,128,17,3,18,6
ENSG00000000005,1,0,0,2,3,0,0,1,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,1495,3271,2303,1234,1896,2013,1745,2403,1847,973,⋯,1751,2537,2459,1269,337,1049,2545,1253,1905,683
ENSG00000000457,231,809,203,1208,425,710,922,1203,1160,587,⋯,493,220,734,771,327,90,406,548,808,393
ENSG00000000460,87,319,31,183,85,174,292,478,218,107,⋯,104,50,267,104,114,17,67,141,199,55
ENSG00000000938,318,105,119,190,2064,12,351,555,37,21,⋯,3064,46,46,14,303,58,77,11,174,22


,GENE_NAME,ALMC1_DJ_p27,ALMC2_DJ_p11,AMO1_DSMZ_p8,ANBL6_DJ2_p13,COLO677_DSMZ_p8,Delta47_JCRB_p16,DP6_DJ_p7,EJM_DSMZ_p9,FLAM76_JCRB_p10,⋯,MMRF_2379_1_BM,MMRF_1474_1_BM,MMRF_1641_1_BM,MMRF_1978_2_BM,MMRF_2795_1_BM,MMRF_2316_1_BM,MMRF_2606_2_BM,MMRF_2440_1_BM,MMRF_2731_1_BM,MMRF_1355_1_BM
ENSG00000000003,TSPAN6,2,0,100,2,0,0,2,88,4,⋯,2,186,102,347,128,17,11,3,18,6
ENSG00000000005,TNMD,1,0,0,0,1,0,0,0,0,⋯,0,0,0,0,0,0,1,0,0,0
ENSG00000000419,DPM1,3949,4134,4795,8278,2262,2587,3607,2450,1388,⋯,2459,1269,337,3347,1049,2545,1625,1253,1905,683
ENSG00000000457,SCYL3,1597,1513,2412,1988,1530,926,1415,573,1013,⋯,734,771,327,392,90,406,857,548,808,393
ENSG00000000460,C1orf112,3072,1488,2193,4091,3208,1170,3614,1618,1507,⋯,267,104,114,126,17,67,196,141,199,55
ENSG00000000938,FGR,0,22,701,5,2,4,89,4,2,⋯,46,14,303,2007,58,77,1037,11,174,22


In [34]:
MM_cell_line_cpm = cpm(MM_cell_line[,-1])
MM_patients_cpm = cpm(MM_patients)

#getting counts per million for MM_cell_line, and MM_patients_cpm

head(MM_cell_line_cpm)
head(MM_patients_cpm)

#checking output head

,ALMC1_DJ_p27,ALMC2_DJ_p11,AMO1_DSMZ_p8,ANBL6_DJ2_p13,COLO677_DSMZ_p8,Delta47_JCRB_p16,DP6_DJ_p7,EJM_DSMZ_p9,FLAM76_JCRB_p10,FR4_p6,⋯,RPMI8226_ATCC_p7,SKMM1_p6,SKMM2_DSMZ_p17,U266_ATCC_p8,UTMC2_p6,VP6_DJ_p5,XG1_p5,XG2_p5,XG6_p5,XG7_p6
ENSG00000000003,0.02834975,0.0000000,1.622395,0.02336384,0.00000000,0.00000000,0.02991395,1.84606143,0.08252468,0.00000,⋯,0.04066095,0.00000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000000,0.03073836,1.460413
ENSG00000000005,0.01417488,0.0000000,0.000000,0.00000000,0.01371330,0.00000000,0.00000000,0.00000000,0.00000000,0.00000,⋯,0.00000000,0.02448497,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000000,0.00000000,0.000000
ENSG00000000419,55.97658401,51.2687903,77.793863,96.70295153,31.01947819,46.41255860,53.94981686,51.39602836,28.63606308,90.65327,⋯,54.77030556,30.53276243,65.452450,53.700359,41.32766,65.86048,109.42511,58.79794048,142.61060806,177.145577
ENSG00000000457,22.63727644,18.7638316,39.132179,23.22366123,20.98134466,16.61307664,21.16412278,12.02037725,20.89937457,33.76204,⋯,13.51976733,24.33806403,9.969313,28.420262,21.31539,19.95552,21.27047,11.85249681,23.66853502,21.265668
ENSG00000000460,43.54521805,18.4537881,35.579133,47.79074350,43.99225731,20.99060439,54.05451570,33.94235669,31.09117223,40.85974,⋯,29.92646243,37.14370537,26.999386,36.482372,41.59361,26.84925,37.61861,30.79260088,12.18775862,8.826533
ENSG00000000938,0.00000000,0.2728383,11.372992,0.05840961,0.02742659,0.07176275,1.33117097,0.08391188,0.04126234,62.23506,⋯,0.00000000,3.06062173,15.585827,0.882069,25.13169,33.59784,22.58310,0.06636336,58.03401832,10.850615


,MMRF_2801_1_BM,MMRF_2718_1_BM,MMRF_2815_1_BM,MMRF_1656_1_BM,MMRF_2461_1_BM,MMRF_2057_1_BM,MMRF_1910_1_BM,MMRF_2341_1_BM,MMRF_2501_1_BM,MMRF_1383_1_BM,⋯,MMRF_1731_1_BM,MMRF_2799_1_BM,MMRF_2379_1_BM,MMRF_1474_1_BM,MMRF_1641_1_BM,MMRF_2795_1_BM,MMRF_2316_1_BM,MMRF_2440_1_BM,MMRF_2731_1_BM,MMRF_1355_1_BM
ENSG00000000003,0.41571843,15.194298,1.9975817,0.18506125,9.58358567,0.2422773,0.2571051,0.57555389,7.5451838,20.2371987,⋯,0.02036196,0.8435951,0.02949247,3.1089737,2.013385,2.8256961,0.2622789,0.05961647,0.2925792,0.1861177
ENSG00000000005,0.02445403,0.000000,0.0000000,0.04112472,0.03589358,0.0000000,0.0000000,0.01338497,0.0000000,0.0000000,⋯,0.00000000,0.0000000,0.00000000,0.0000000,0.000000,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000
ENSG00000000419,36.55876793,49.750299,54.1227124,25.37395378,22.68474210,40.6420105,28.0405222,32.16409293,23.4612029,28.4138446,⋯,35.65378598,43.6775686,36.26098596,21.2112241,6.652067,23.1574629,39.2646952,24.89981253,30.9646322,21.1863986
ENSG00000000457,5.64887986,12.304492,4.7706950,24.83933239,5.08492373,14.3347379,14.8156799,16.10212393,14.7347024,17.1417541,⋯,10.03844460,3.7875700,10.82373473,12.8871976,6.454676,1.9868176,6.2638374,10.88994195,13.1335553,12.1907096
ENSG00000000460,2.12750021,4.851833,0.7285298,3.76291211,1.01698475,3.5130203,4.6921676,6.39801765,2.7691079,3.1246468,⋯,2.11764348,0.8608114,3.93724410,1.7383509,2.250254,0.3752878,1.0336875,2.80197412,3.2346256,1.7060790
ENSG00000000938,7.77638007,1.596998,2.7966143,3.90684864,24.69478254,0.2422773,5.6402426,7.42866066,0.4699862,0.6132484,⋯,62.38903497,0.7919465,0.67832670,0.2340088,5.980938,1.2803936,1.1879692,0.21859373,2.8282656,0.6824316


In [35]:
MM_patients_trans = data.frame(fread("IA13a.structural.variants.annot.translocations.clean.maxHom0.8.minMap0.2.clonality.readRange10000bp.cnvRange2000bp.common.txt"))
MM_patient_trans_ids = MM_patients_trans[,'sample']
keepMYC = as.logical(MM_patients_trans[,'t.MYC.'])
MM_patients_trans_MYC = MM_patient_trans_ids[keepMYC]
temp = colnames(MM_patients) %in% MM_patients_trans_MYC
MM_patients_trans_MYC = MM_patients[,temp]
head(MM_patients_trans_MYC)
MM_combined_MYC = 
#colnames(MM_patients_trans_MYC) %in% MM_patient_trans_ids[keepMYC]
#length(MM_patients_trans_MYC[2,])
#head(temp)
#sum(temp)
#sum(keepMYC)
#length(unique(MM_patient_trans_ids[keepMYC]))
#MM_patients_trans_MYC = unique(MM_patients_trans_MYC)
#MM_patients_trans_MYC = MM_patients[,MM_patients_trans_MYC]
#head(MM_patients_trans_MYC)

,MMRF_1656_1_BM,MMRF_2501_1_BM,MMRF_1383_1_BM,MMRF_1466_1_BM,MMRF_2400_1_BM,MMRF_2549_1_BM,MMRF_1423_1_BM,MMRF_1328_1_BM,MMRF_2445_1_BM,MMRF_2286_1_BM,⋯,MMRF_2574_1_BM,MMRF_1876_1_BM,MMRF_1862_1_BM,MMRF_2118_1_BM,MMRF_1510_1_BM,MMRF_2498_1_BM,MMRF_1274_1_BM,MMRF_2265_1_BM,MMRF_1641_1_BM,MMRF_2440_1_BM
ENSG00000000003,9,594,693,182,1,34,140,2,162,537,⋯,1360,402,607,18,437,275,16,45,102,3
ENSG00000000005,2,0,0,1,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,1234,1847,973,1426,2759,1369,878,2437,2008,1698,⋯,2647,1925,5411,3482,2036,1941,1402,1723,337,1253
ENSG00000000457,1208,1160,587,625,650,1883,381,1870,694,624,⋯,1465,917,478,1510,780,869,978,908,327,548
ENSG00000000460,183,218,107,128,229,350,74,606,297,66,⋯,597,194,125,311,100,100,233,204,114,141
ENSG00000000938,190,37,21,25,124,41,9,70,259,35,⋯,137,50,320,132,43,10,138,113,303,11


In [36]:
keep414 = as.logical(MM_patients_trans[,'t.4.14.'])
MM_patients_trans_414 = MM_patient_trans_ids[keep414]
temp = colnames(MM_patients) %in% MM_patients_trans_414
MM_patients_trans_414 = MM_patients[,temp]
head(MM_patients_trans_414)
length(MM_patients_trans_414[2,])

,MMRF_2815_1_BM,MMRF_2636_1_BM,MMRF_1572_1_BM,MMRF_1391_1_BM,MMRF_1785_1_BM,MMRF_2621_1_BM,MMRF_2203_1_BM,MMRF_2694_1_BM,MMRF_2238_1_BM,MMRF_1627_1_BM,⋯,MMRF_1497_1_BM,MMRF_1324_1_BM,MMRF_1216_1_BM,MMRF_1359_1_BM,MMRF_2220_1_BM,MMRF_1613_1_BM,MMRF_2272_1_BM,MMRF_2176_1_BM,MMRF_2118_1_BM,MMRF_2799_1_BM
ENSG00000000003,85,472,820,456,899,1,1537,826,2,256,⋯,481,5,0,2,1652,6,116,2,18,49
ENSG00000000005,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,2303,2057,1618,902,1466,2504,2366,1519,1772,1797,⋯,1802,1686,928,1455,1736,885,2408,1327,3482,2537
ENSG00000000457,203,591,974,592,759,1653,315,626,584,967,⋯,356,895,1231,1511,856,753,1119,398,1510,220
ENSG00000000460,31,48,363,162,306,1094,150,261,126,362,⋯,56,242,348,955,109,217,557,104,311,50
ENSG00000000938,119,42,55,20,12,446,26,100,307,170,⋯,41,54,36,18,14,4,719,17,132,46


[1] 71

In [37]:
keep1114 = as.logical(MM_patients_trans[,'t.11.14.'])
MM_patients_trans_1114 = MM_patient_trans_ids[keep1114]
temp = colnames(MM_patients) %in% MM_patients_trans_1114
MM_patients_trans_1114 = MM_patients[,temp]
head(MM_patients_trans_1114)
length(MM_patients_trans_1114[2,])

,MMRF_2801_1_BM,MMRF_2718_1_BM,MMRF_1439_1_BM,MMRF_1261_1_BM,MMRF_2173_1_BM,MMRF_1865_1_BM,MMRF_2639_1_BM,MMRF_2140_1_BM,MMRF_1886_1_BM,MMRF_1787_1_BM,⋯,MMRF_2654_1_BM,MMRF_2201_1_BM,MMRF_1556_1_BM,MMRF_1580_1_BM,MMRF_1542_1_BM,MMRF_1773_1_BM,MMRF_1286_1_BM,MMRF_2499_1_BM,MMRF_1862_1_BM,MMRF_1539_1_BM
ENSG00000000003,17,999,2,6,19,10,207,12,18,2,⋯,7,62,20,991,14,8,809,69,607,1615
ENSG00000000005,1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,1,0,0
ENSG00000000419,1495,3271,2486,1882,2031,1273,1844,2266,1302,1959,⋯,3154,3107,4159,2532,2822,4693,2827,1052,5411,2242
ENSG00000000457,231,809,884,1511,496,583,215,755,1088,1432,⋯,221,1139,1234,1033,945,745,1207,494,478,1081
ENSG00000000460,87,319,398,428,165,122,121,200,324,299,⋯,56,542,402,422,213,223,436,162,125,475
ENSG00000000938,318,105,53,30,22,5,243,51,65,15,⋯,41,90,350,311,232,62,39,548,320,17


[1] 109

In [38]:
keep1420 = as.logical(MM_patients_trans[,'t.14.20.'])
MM_patients_trans_1420 = MM_patient_trans_ids[keep1420]
temp = colnames(MM_patients) %in% MM_patients_trans_1420
MM_patients_trans_1420 = MM_patients[,temp]
head(MM_patients_trans_1420)
length(MM_patients_trans_1420[2,])
#no need to go further

,MMRF_1830_1_BM,MMRF_2076_1_BM,MMRF_1461_1_BM,MMRF_1625_1_BM,MMRF_1778_1_BM,MMRF_2153_1_BM,MMRF_1832_1_BM
ENSG00000000003,1,689,10,154,3,381,3
ENSG00000000005,0,0,0,0,0,1,0
ENSG00000000419,1862,1437,1882,894,2496,1471,1686
ENSG00000000457,1213,657,757,488,2156,763,1012
ENSG00000000460,177,281,202,199,737,169,379
ENSG00000000938,33,11,18,23,3,15776,68


[1] 7

In [39]:
keep1416 = as.logical(MM_patients_trans[,'t.14.16.'])
MM_patients_trans_1416 = MM_patient_trans_ids[keep1416]
temp = colnames(MM_patients) %in% MM_patients_trans_1416
MM_patients_trans_1416 = MM_patients[,temp]
head(MM_patients_trans_1416)
length(MM_patients_trans_1416[2,])

,MMRF_1328_1_BM,MMRF_2041_1_BM,MMRF_1912_1_PB,MMRF_1491_1_BM,MMRF_1965_1_PB,MMRF_1403_1_BM,MMRF_1965_1_BM,MMRF_1683_1_BM,MMRF_1164_1_BM,MMRF_2306_1_BM,⋯,MMRF_1290_1_BM,MMRF_1454_1_BM,MMRF_1618_1_BM,MMRF_1586_1_BM,MMRF_2695_1_BM,MMRF_1855_1_BM,MMRF_1180_1_BM,MMRF_2794_1_BM,MMRF_2269_1_BM,MMRF_1971_1_BM
ENSG00000000003,2,12,0,9,6,0,3,622,385,454,⋯,2,3,970,1,81,823,3,1,1,5
ENSG00000000005,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000000419,2437,3589,3083,4532,1486,1508,2120,1249,1535,2333,⋯,878,1117,1188,1788,3382,1712,1989,6024,2834,2990
ENSG00000000457,1870,1120,1243,899,1490,750,1483,1272,644,491,⋯,911,625,618,1386,507,1093,585,232,287,1735
ENSG00000000460,606,531,276,336,399,126,495,361,262,67,⋯,222,137,217,458,87,333,51,42,198,573
ENSG00000000938,70,21,1083,102,146,32,37,29,1358,26,⋯,22,115,13,65,46,15,107,130,19,1857


[1] 27

In [ ]:
MM_cell_line_cpm = cpm(MM_cell_line[,-1])
MM_patients_cpm = cpm(MM_patients)

#getting counts per million for MM_cell_line, and MM_patients_cpm

head(MM_cell_line_cpm)
head(MM_patients_cpm)

#checking output head

In [ ]:
keep = rowSums(cpm(MM_combined[-1]) > 1) > 2
MM_combined.filtered = MM_combined[keep, ]
MM_patients_cpm = MM_patients_cpm[keep, ]

#initial filtering of MM_combined to remove low count genes (genes with greater than 
    #1 million counts in more than 2 samples are kept)

head(keep)
head(MM_combined.filtered)

#checking output head

In [ ]:
MM_gene_names = MM_combined.filtered[, 1]
MM_combined.filtered = MM_combined.filtered[, -1]

#grabbing first colunmn of MM_combined.filtered (Gene_name), only those rows specified by ensembl intersect, 
    #removing Gene_name column from MM_combined.filtered

head(MM_gene_names)
head(MM_combined.filtered)

In [ ]:
combined_annot = data.frame(colnames(MM_combined.filtered))
combined_annot$type = "Patient"
combined_annot$type[1:ncol(MM_cell_line[-1])] = "Cell_Line"
rn = combined_annot[,1]
combined_annot = combined_annot[-1]
rownames(combined_annot) = rn
setv = vst(as.matrix(round(MM_combined.filtered)))
setv = newSeqExpressionSet(setv, phenoData = combined_annot)

#utilizing varaince stabilization normalization method, vst(), then creating new seqExpressionSet object, 
    #using normalized counts

symbols = c(rep(2, ncol(MM_cell_line[-1])), rep(1, ncol(MM_patients)))
plotRLE(setv, outline=FALSE, ylim=c(-4, 4))
affycoretools::plotPCA(setv, groupnames = c('Cell_Line', 'Patient'), 
                       pch = symbols, col = symbols, outside = TRUE)

#plotting RLE to check for batch effects, still present but presumption is that VST makes genes more comparable

In [ ]:
keepi = apply(MM_patients_cpm, 1, IQR)

head(keepi)

keepi = sort(keepi, decreasing = TRUE)

tail(keepi[1:5000])

keepi = apply(MM_patients_cpm, 1, IQR)
keepi = keepi > 17
head(keepi)

In [ ]:
MM_combined.filtered = MM_combined.filtered[keepi, ]

head(MM_combined.filtered)

In [27]:
MM_patients_cpm_MYC = cpm()

ERROR: Error in as.matrix(y): argument "y" is missing, with no default
